### Variable Threshold

In [33]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device='cuda'


### original inference process ###
stmodel= tm.resnet50().to(device)
stmodel = DataParallel(stmodel)
stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform_train, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
atk = torchattacks.PGD(stmodel, eps=4/255, alpha=1/255, steps=1)
radius = 1
n_points = 8 * radius
stmodel.eval()
print('Start validating')
t = trange(len(val_loader), desc='Average Val Accuracy', leave=True)
val_acc, val_acc2 = [], []
for step_val, (feat,X,y) in zip(t, val_loader):
    X = X.to(device)
    y = y.to(device)
    c=feat.to(device)
    ##======================

    adv_images1 = atk(X/255.0, y)
    adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255
    
    for b in range(adv_images.shape[0]):
        adv_images[b] = val_ltp(image=adv_images[b])['image']
#         for c in range(3):
#             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')
            
#     adv_images[np.isnan(adv_images)] = 0
#     adv_images = (adv_images / float(2**8) -0.5) * 2
    adv_images = (adv_images*0.125 -0.5) * 2
    
    adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
    
    outputs = stmodel(adv_images)
    outputs2 = stmodel(feat)
    _, pred = torch.max(outputs.data, 1)
    val_acc2.append( (pred == y).sum().item()/y.size(0) )
    _, pred2 = torch.max(outputs2.data, 1)
    val_acc.append( (pred2 == y).sum().item()/y.size(0) )

#     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
#     plt.show()
    t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
    t.refresh() # to show immediately the update
#     break

The # of images is: 50000 on val mode!
Start validating


Average Val Accuracy:   0%|                                                                                                                                                                                         | 0/2084 [00:00<?, ?it/s]/home/jess/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Average Val Accuracy: 0.65403/Robustness: 0.62486: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [24:47<00:00,  1.40it/s]


### fixed threshold

In [34]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device='cuda'


### original inference process ###
stmodel= tm.resnet50().to(device)
stmodel = DataParallel(stmodel)
stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
atk = torchattacks.PGD(stmodel, eps=4/255, alpha=1/255, steps=1)
radius = 1
n_points = 8 * radius
stmodel.eval()
print('Start validating')
t = trange(len(val_loader), desc='Average Val Accuracy', leave=True)
val_acc, val_acc2 = [], []
for step_val, (feat,X,y) in zip(t, val_loader):
    X = X.to(device)
    y = y.to(device)
    c=feat.to(device)
    ##======================

    adv_images1 = atk(X/255.0, y)
    adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255
    
    for b in range(adv_images.shape[0]):
        adv_images[b] = val_ltp(image=adv_images[b])['image']
#         for c in range(3):
#             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')
            
#     adv_images[np.isnan(adv_images)] = 0
#     adv_images = (adv_images / float(2**8) -0.5) * 2
    adv_images = (adv_images*0.125 -0.5) * 2
    
    adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
    
    outputs = stmodel(adv_images)
    outputs2 = stmodel(feat)
    _, pred = torch.max(outputs.data, 1)
    val_acc2.append( (pred == y).sum().item()/y.size(0) )
    _, pred2 = torch.max(outputs2.data, 1)
    val_acc.append( (pred2 == y).sum().item()/y.size(0) )

#     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
#     plt.show()
    t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
    t.refresh() # to show immediately the update
#     break

The # of images is: 50000 on val mode!
Start validating


Average Val Accuracy:   0%|                                                                                                                                                                                         | 0/2084 [00:00<?, ?it/s]/home/jess/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Average Val Accuracy: 0.65403/Robustness: 0.62474: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [25:25<00:00,  1.37it/s]


In [35]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device='cuda'


### original inference process ###
stmodel= tm.resnet50().to(device)
stmodel = DataParallel(stmodel)
stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
atk = torchattacks.PGD(stmodel, eps=4/255, alpha=1/255, steps=10)
radius = 1
n_points = 8 * radius
stmodel.eval()
print('Start validating')
t = trange(len(val_loader), desc='Average Val Accuracy', leave=True)
val_acc, val_acc2 = [], []
for step_val, (feat,X,y) in zip(t, val_loader):
    X = X.to(device)
    y = y.to(device)
    c=feat.to(device)
    ##======================

    adv_images1 = atk(X/255.0, y)
    adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255
    
    for b in range(adv_images.shape[0]):
        adv_images[b] = val_ltp(image=adv_images[b])['image']
#         for c in range(3):
#             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')
            
#     adv_images[np.isnan(adv_images)] = 0
#     adv_images = (adv_images / float(2**8) -0.5) * 2
    adv_images = (adv_images*0.125 -0.5) * 2
    
    adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
    
    outputs = stmodel(adv_images)
    outputs2 = stmodel(feat)
    _, pred = torch.max(outputs.data, 1)
    val_acc2.append( (pred == y).sum().item()/y.size(0) )
    _, pred2 = torch.max(outputs2.data, 1)
    val_acc.append( (pred2 == y).sum().item()/y.size(0) )

#     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
#     plt.show()
    t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
    t.refresh() # to show immediately the update
#     break

The # of images is: 50000 on val mode!
Start validating


Average Val Accuracy:   0%|                                                                                                                                                                                         | 0/2084 [00:00<?, ?it/s]/home/jess/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Average Val Accuracy: 0.65391/Robustness: 0.60307: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [1:54:00<00:00,  3.28s/it]


In [36]:
import cv2
import os, glob
import torch.nn as nn
import torch
import numpy as np
from dataset import *
from model import *
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as tm
from matplotlib import pyplot as plt
import torchvision
from skimage.feature import local_binary_pattern
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device='cuda'


### original inference process ###
stmodel= tm.resnet50().to(device)
stmodel = DataParallel(stmodel)
stmodel.load_state_dict(torch.load('checkpoint2/final_hog.pt'))
val_ltp = A.Compose([A.Lambda(name='ltp', image=ltp_transform, p=1, always_apply=True)])


val_dataset = featDataset(0, "val.txt", root='/hdd3/ILSVRC/Data/imagenet/', mode='val', feat='ltp')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=24, drop_last=False, 
                                         num_workers=16, sampler=None, shuffle=True)
    
atk = torchattacks.PGD(stmodel, eps=4/255, alpha=1/255, steps=50)
radius = 1
n_points = 8 * radius
stmodel.eval()
print('Start validating')
t = trange(len(val_loader), desc='Average Val Accuracy', leave=True)
val_acc, val_acc2 = [], []
for step_val, (feat,X,y) in zip(t, val_loader):
    X = X.to(device)
    y = y.to(device)
    c=feat.to(device)
    ##======================

    adv_images1 = atk(X/255.0, y)
    adv_images = adv_images1.permute(0,2,3,1).cpu().numpy().astype(np.float)*255
    
    for b in range(adv_images.shape[0]):
        adv_images[b] = val_ltp(image=adv_images[b])['image']
#         for c in range(3):
#             adv_images[b,:,:,c] =local_binary_pattern(adv_images[b,:,:,c], n_points, radius, 'ror')
            
#     adv_images[np.isnan(adv_images)] = 0
#     adv_images = (adv_images / float(2**8) -0.5) * 2
    adv_images = (adv_images*0.125 -0.5) * 2
    
    adv_images = torch.Tensor(adv_images).permute(0,3,1,2).cuda()
    
    outputs = stmodel(adv_images)
    outputs2 = stmodel(feat)
    _, pred = torch.max(outputs.data, 1)
    val_acc2.append( (pred == y).sum().item()/y.size(0) )
    _, pred2 = torch.max(outputs2.data, 1)
    val_acc.append( (pred2 == y).sum().item()/y.size(0) )

#     plt.imshow(adv_images[0].permute(1,2,0).cpu().data)
#     plt.show()
    t.set_description("Average Val Accuracy: %.5f/Robustness: %.5f" % (np.mean(np.array(val_acc)), np.mean(np.array(val_acc2))))
    t.refresh() # to show immediately the update
#     break

The # of images is: 50000 on val mode!
Start validating


Average Val Accuracy:   0%|                                                                                                                                                                                         | 0/2084 [00:00<?, ?it/s]/home/jess/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Average Val Accuracy: 0.65407/Robustness: 0.59927: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2084/2084 [2:56:08<00:00,  5.07s/it]
